# Scrapper letterboxd

WEb scrapping para descargar lista de peliculas del sitio de letterboxd.com
- se busca una pelicula
- se linkea en el sitio
- se descarga todas las peliculas de la lista

In [ ]:
import requests

URL = "https://letterboxd.com/film/the-godfather/lists/by/popular/"
page = requests.get(URL)

print(page.text)

# DESCARGAR 10 PAGINAS DE LISTA DE UNA SOLA PELICULA

In [ ]:
import requests
from bs4 import BeautifulSoup
import csv



def scrapear(nombre_pelicula, id_pelicula):
    # Lista para almacenar los IDs de las listas
    lista_ids = []

    # Configuración de encabezados para la solicitud HTTP
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/66.0.3359.181 Safari/537.36'
    }

    # Iterar sobre las páginas 1 a 10
    for pagina in range(1, 11):
        if pagina == 1:
            url = f"https://letterboxd.com/film/{nombre_pelicula}/lists/by/popular/"
        else:
            url = f"https://letterboxd.com/film/{nombre_pelicula}/lists/by/popular/page/{pagina}/"

        # Obtener el contenido HTML de la página
        response = requests.get(url, headers=headers)
        if response.status_code != 200:
            print(f"Error al acceder a la página {pagina}: {response.status_code}")
            continue

        html = response.content

        # Parsear el HTML con BeautifulSoup
        soup = BeautifulSoup(html, 'html.parser')

        # Encontrar todos los elementos <section> con las clases especificadas
        secciones = soup.select('section.list.-overlapped.-summary')

        for seccion in secciones:
            # Extraer el atributo 'data-film-list-id'
            list_id = seccion.get('data-film-list-id')
            if list_id:
                lista_ids.append(list_id)

    # Exportar los resultados a un archivo CSV
    with open('movies-list.csv', mode='w', newline='', encoding='utf-8') as archivo_csv:
        escritor_csv = csv.writer(archivo_csv)
        # Escribir el encabezado
        escritor_csv.writerow(['ID_Pelicula', 'ID_Lista'])
        # Escribir los datos
        for id_lista in lista_ids:
            escritor_csv.writerow([id_pelicula, id_lista])

    print("Proceso completado. Los resultados se han guardado en 'movies-list.csv'.")


In [ ]:
# Datos de entrada
nombre_pelicula = "the-godfather"  # Formato para la URL (en minúsculas y con guiones)
id_pelicula = "238"

scrapear(nombre_pelicula, id_pelicula)

# GUARDAMOS UN CSV CON EL MATCH ID_PELICULA E ID_LISTA, SE GENERA UN CSV POR CADA PELICULA DEL CSV "MOVIE.CSV"


In [ ]:
import requests
from bs4 import BeautifulSoup
import csv
import pandas as pd
import os

def scrapear(original_title, id_pelicula):
    # Formatear el nombre de la película para la URL
    nombre_pelicula = original_title.lower().replace(' ', '-')
    
    # Lista para almacenar los IDs de las listas
    lista_ids = []
    
    # Configuración de encabezados para la solicitud HTTP
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/66.0.3359.181 Safari/537.36'
    }
    
    # Iterar sobre las páginas 1 a 10
    for pagina in range(1, 11):
        if pagina == 1:
            url = f"https://letterboxd.com/film/{nombre_pelicula}/lists/by/popular/"
        else:
            url = f"https://letterboxd.com/film/{nombre_pelicula}/lists/by/popular/page/{pagina}/"
        
        # Obtener el contenido HTML de la página
        response = requests.get(url, headers=headers)
        if response.status_code != 200:
            print(f"Error al acceder a la página {pagina} para '{original_title}': {response.status_code}")
            continue
        
        html = response.content
        
        # Parsear el HTML con BeautifulSoup
        soup = BeautifulSoup(html, 'html.parser')
        
        # Encontrar todos los elementos <section> con las clases especificadas
        secciones = soup.select('section.list.-overlapped.-summary')
        
        for seccion in secciones:
            # Extraer el atributo 'data-film-list-id'
            list_id = seccion.get('data-film-list-id')
            if list_id:
                lista_ids.append(list_id)
    
    # Exportar los resultados a un archivo CSV
    # Crear el directorio 'resultados' si no existe
    if not os.path.exists('resultados'):
        os.makedirs('resultados')
    
    # Nombre del archivo CSV para esta película
    nombre_archivo = f"resultados/{id_pelicula}_{nombre_pelicula}.csv"
    
    with open(nombre_archivo, mode='w', newline='', encoding='utf-8') as archivo_csv:
        escritor_csv = csv.writer(archivo_csv)
        # Escribir el encabezado
        escritor_csv.writerow(['ID_Pelicula', 'ID_Lista'])
        # Escribir los datos
        for id_lista in lista_ids:
            escritor_csv.writerow([id_pelicula, id_lista])
    
    print(f"Proceso completado para '{original_title}'. Los resultados se han guardado en '{nombre_archivo}'.")



In [ ]:

# Leer el archivo CSV 'movies.csv'
movies_df = pd.read_csv('movies.csv')
count =1
# Iterar sobre cada fila y llamar a la función 'scrape'
for index, row in movies_df.iterrows():
    if count <= 2:
        id_pelicula = str(row['id'])
        original_title = str(row['original_title'])
        scrapear(original_title, id_pelicula)
    else:
        exit()

# GUARDAR TODAS LAS SALIDAS EN UN SOLO CSV

In [ ]:
import requests
from bs4 import BeautifulSoup
import csv
import pandas as pd
from unidecode import unidecode
import os

def scrape(original_title, id_pelicula, processed_movies, all_pairs):
    # Si la película ya fue procesada, saltarla
    if id_pelicula in processed_movies:
        print(f"'{original_title}' (ID: {id_pelicula}) ya fue procesada. Saltando.")
        return all_pairs

    # Formatear el nombre de la película para la URL
    nombre_pelicula = unidecode(original_title.lower()).replace(' ', '-').replace("'", '')
    nombre_pelicula = ''.join(c for c in nombre_pelicula if c.isalnum() or c == '-')
    
    # Set para almacenar los pares únicos (id_pelicula, id_lista)
    lista_ids = set()
    
    # Configuración de encabezados para la solicitud HTTP
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/66.0.3359.181 Safari/537.36'
    }
    
    # Iterar sobre las páginas 1 a 10
    for pagina in range(1, 11):
        if pagina == 1:
            url = f"https://letterboxd.com/film/{nombre_pelicula}/lists/by/popular/"
        else:
            url = f"https://letterboxd.com/film/{nombre_pelicula}/lists/by/popular/page/{pagina}/"
        
        # Obtener el contenido HTML de la página
        try:
            response = requests.get(url, headers=headers)
            if response.status_code != 200:
                print(f"Error al acceder a la página {pagina} para '{original_title}': {response.status_code}")
                continue
        except requests.exceptions.RequestException as e:
            print(f"Excepción al acceder a la página {pagina} para '{original_title}': {e}")
            continue
        
        html = response.content
        
        # Parsear el HTML con BeautifulSoup
        soup = BeautifulSoup(html, 'html.parser')
        
        # Encontrar todos los elementos <section> con las clases especificadas
        secciones = soup.select('section.list.-overlapped.-summary')
        
        for seccion in secciones:
            # Extraer el atributo 'data-film-list-id'
            list_id = seccion.get('data-film-list-id')
            if list_id:
                # Agregar el par al set para evitar duplicados
                lista_ids.add((id_pelicula, list_id))
    
    # Actualizar el conjunto total de pares
    all_pairs.update(lista_ids)
    
    # Agregar el ID de la película al conjunto de películas procesadas
    processed_movies.add(id_pelicula)
    
    # Guardar los resultados actuales en el archivo CSV
    save_results(lista_ids)
    
    print(f"Proceso completado para '{original_title}'. {len(lista_ids)} pares agregados.")
    
    return all_pairs

def save_results(pairs):
    # Verificar si el archivo ya existe
    file_exists = os.path.isfile('resultados.csv')
    
    # Abrir el archivo en modo adjuntar
    with open('resultados.csv', mode='a', newline='', encoding='utf-8') as archivo_csv:
        escritor_csv = csv.writer(archivo_csv)
        # Escribir el encabezado solo si el archivo no existe
        if not file_exists:
            escritor_csv.writerow(['ID_Pelicula', 'ID_Lista'])
        # Escribir los datos
        for id_pelicula, id_lista in pairs:
            escritor_csv.writerow([id_pelicula, id_lista])

def load_processed_movies():
    processed_movies = set()
    all_pairs = set()
    if os.path.isfile('resultados.csv'):
        with open('resultados.csv', mode='r', newline='', encoding='utf-8') as archivo_csv:
            lector_csv = csv.reader(archivo_csv)
            next(lector_csv, None)  # Saltar el encabezado
            for row in lector_csv:
                id_pelicula, id_lista = row
                all_pairs.add((id_pelicula, id_lista))
                processed_movies.add(id_pelicula)
    return processed_movies, all_pairs

# Cargar el archivo CSV 'movies.csv'
movies_df = pd.read_csv('movies.csv')

# Cargar las películas ya procesadas y los pares existentes
processed_movies, all_pairs = load_processed_movies()

movieCount =1
# Iterar sobre cada fila y llamar a la función 'scrape'
for index, row in movies_df.iterrows():
    id_pelicula = str(row['id'])
    original_title = str(row['original_title'])
    print(f" {movieCount} | Procesando '{original_title}' (ID: {id_pelicula})")
    all_pairs = scrape(original_title, id_pelicula, processed_movies, all_pairs)
    movieCount+=1
print("Proceso completado. Los resultados se han guardado en 'resultados.csv'.")


 1 | Procesando 'Avatar' (ID: 19995)
'Avatar' (ID: 19995) ya fue procesada. Saltando.
 2 | Procesando 'Pirates of the Caribbean: At World's End' (ID: 285)
'Pirates of the Caribbean: At World's End' (ID: 285) ya fue procesada. Saltando.
 3 | Procesando 'Spectre' (ID: 206647)
'Spectre' (ID: 206647) ya fue procesada. Saltando.
 4 | Procesando 'The Dark Knight Rises' (ID: 49026)
'The Dark Knight Rises' (ID: 49026) ya fue procesada. Saltando.
 5 | Procesando 'John Carter' (ID: 49529)
'John Carter' (ID: 49529) ya fue procesada. Saltando.
 6 | Procesando 'Spider-Man 3' (ID: 559)
'Spider-Man 3' (ID: 559) ya fue procesada. Saltando.
 7 | Procesando 'Tangled' (ID: 38757)
'Tangled' (ID: 38757) ya fue procesada. Saltando.
 8 | Procesando 'Avengers: Age of Ultron' (ID: 99861)
'Avengers: Age of Ultron' (ID: 99861) ya fue procesada. Saltando.
 9 | Procesando 'Harry Potter and the Half-Blood Prince' (ID: 767)
'Harry Potter and the Half-Blood Prince' (ID: 767) ya fue procesada. Saltando.
 10 | Procesan

In [3]:
import numpy as np
import pandas as pd
resultados_df = pd.read_csv("resultados.csv")

In [4]:
pd.set_option('display.max_rows', None)
resultados_df

,ID_Pelicula,ID_Lista
0,285,2457830
1,19995,41864669
2,19995,43478626
3,285,18477098
4,19995,206047
5,285,18527180
6,285,1895534
7,19995,17132725
8,285,301178
9,285,39936
